In [ ]:
import wandb

In [ ]:
import wandb
import pandas as pd

# Initialize the W&B API
api = wandb.Api()

# Replace with your entity, project, and run ID
# Access the run
# run = api.run(f"{entity}/{project}/{run_id}")
run = api.run("rmit_is/imagenet_c_vision_transformers/eyrvah10")

# Retrieve the run's history
history = run.scan_history()

# Convert the history to a DataFrame
df = pd.DataFrame(history)

# Save the DataFrame to a CSV file
df.to_csv('run_history.csv', index=False)

print(df.columns)

In [ ]:
corruptions =["gaussian_noise",
    "shot_noise",
    "impulse_noise",
    "defocus_blur",
    "glass_blur",
    "motion_blur",
    "zoom_blur",
    "snow",
    "frost",
    "fog",
    "brightness",
    "contrast",
    "elastic_transform",
    "pixelate",
    "jpeg_compression"]

display_df = pd.DataFrame(columns=["measurements"] + corruptions)
display_df["measurements"] = ["Max Accuracy", "Avg Accuracy", "Unadapted Accuracy"]
for corruption in corruptions:
    sub_df = df.dropna(subset=[f"avg_accuracy/{corruption}"])
    sub_df = sub_df.sort_values(by='custom_step', ascending=True)
    
    avg_acc = 0
    max_all_class_acc = -1

    for ii in range(len(sub_df)):
        all_class_acc = sub_df.iloc[ii][f"avg_accuracy/{corruption}"]
        step = sub_df.iloc[ii]["custom_step"]

        if step == 0:
            continue
        avg_acc += all_class_acc

    unadapted_acc = sub_df.iloc[0][f"avg_accuracy/{corruption}"]
    avg_acc /= len(sub_df)-1
    max_all_class_acc = sub_df[1:][f"avg_accuracy/{corruption}"].max()
    

    display_df.loc[0, corruption] = max_all_class_acc.round(4)
    display_df.loc[1, corruption] = avg_acc.round(4)
    display_df.loc[2, corruption] = unadapted_acc.round(4)
    
    print(f"Corruption: {corruption}, Avg Accuracy: {avg_acc:.4f} \t Max Accuracy: {max_all_class_acc:.4f}")


In [ ]:
display_df